## Data Mining Assignment 6
### Author G. Gordon
### Lecturer  Dr. Pawan Lingras
March 6, 2016

## Data Preparation

An archive available at (http://dev.cs.smu.ca/~pawan/5580/notes/week08/textMining/med/med.all.Z) was unzipped to retrieve a file named 'med.all'

The following Java code was used to split the aforementioned file into separate files .

```

import java.util.regex.Pattern;
import java.io.*;

/**
 * @author ggordon
 * @created 6.3.2016
 */
public class FileSeparator {

	private final static String DIR = "medCollection";

	/**
	 * @param args
	 */
	public static void main(String[] args) {
		System.out.println("Received : " + args.length + " args");
		if (args.length == 0) {
             //use default file
			args = new String[]{"dm06/med.all"};
		}
		for (String arg : args) {
			System.out.println("Parsing File : " + arg);
			parseFile(arg);
		}
        System.out.println("Parsing Complete");
	}

	private static void parseFile(String fileName) {
		Pattern newStoryPattern = Pattern.compile("\\.I [0-9]+");
		try {
			verifyDirectoryExists();
			File f = new File(fileName);

			BufferedReader br = new BufferedReader(new FileReader(f));

			String line = null, story = "", filename = null;
			FileWriter tempStoryFile = null;

			while ((line = br.readLine()) != null) {

				// verify whether new file should start
				if (newStoryPattern.matcher(line).find()) {
					saveStory(tempStoryFile,story,filename);
					// refresh buffers
					story = "";
					
					// get filename
					filename = DIR + File.separator + line.split(" ")[1] + ".txt";
					
					// create new file write
					tempStoryFile = new FileWriter(filename);

					// skip .I and .W line
					line = br.readLine();
					line = br.readLine();

				}
				story += line + "\n";
			}
			saveStory(tempStoryFile,story,filename);
			br.close();
		} catch (Exception e) {
			e.printStackTrace(System.err);
		}
	}
	
	private static void saveStory(FileWriter tempStoryFile,String story, String filename) throws IOException{
		if(tempStoryFile != null){
			tempStoryFile.write(story);
			System.out.println("Created "+filename);
		    tempStoryFile.close();
		}
	}

	private static boolean verifyDirectoryExists() {
		try {
			File dir = new File(DIR);
			if (dir.isDirectory())
				return true;
			return dir.mkdir();
		} catch (Exception e) {
			e.printStackTrace(System.err);
			return false;
		}
	}

}

```

## Data Analyis

In [1]:
library(tm)

Loading required package: NLP


In [21]:
medCollection=Corpus(DirSource("medCollection"))
medCollection[["2.txt"]]
# inspect(medCollection)

<<PlainTextDocument>>
Metadata:  7
Content:  chars: 1562

In [22]:
medCollection=tm_map(medCollection,removePunctuation)

In [24]:
medCollection=tm_map(medCollection,stripWhitespace)

In [19]:
#medCollection=tm_map(medCollection,tolower)

In [25]:
medCollection=tm_map(medCollection,removeWords,stopwords("english"))

In [27]:
medCollection=tm_map(medCollection,stemDocument)

In [12]:
#inspect(medCollection)
medCollection["2.txt"]

<<VCorpus>>
Metadata:  corpus specific: 0, document level (indexed): 0
Content:  documents: 1

In [28]:
dtmMedCollection = DocumentTermMatrix(medCollection)

In [30]:
#inspect(dtmMedCollection[,30:35])

In [32]:
head(findFreqTerms(dtmMedCollection,5),45)

[1] "100"                     "1000"                   
 [3] "116"                     "120"                    
 [5] "125"                     "131"                    
 [7] "150"                     "1500"                   
 [9] "168"                     "17hydroxycorticosteroid"
[11] "17ketosteroid"           "17ohc"                  
[13] "17oxosteroid"            "1959"                   
[15] "1960"                    "1961"                   
[17] "1962"                    "1963"                   
[19] "1964"                    "1st"                    
[21] "200"                     "250"                    
[23] "2nd"                     "300"                    
[25] "3rd"                     "4th"                    
[27] "500"                     "5fluorouracil"          
[29] "5year"                   "600"                    
[31] "6mercaptopurin"          "6phosphat"              
[33] "99m"                     "abdomin"                
[35] "aberr"                   "abil"                   
[37] "abl"                     "abnorm"                 
[39] "abolish"                 "abort"                  
[41] "absenc"                  "absent"                 
[43] "absolut"                 "absorb"                 
[45] "absorpt"

In [33]:
findAssocs(dtmMedCollection,"data",0.9)

$data
numeric(0)

In [38]:
findAssocs(dtmMedCollection,"data",0.25)

$data
            miner              1770             alloc              appa 
             0.30              0.28              0.28              0.28 
          azotaem               bel bloodureanitrogen            calcem 
             0.28              0.28              0.28              0.28 
           commit       curvilinear         cytokinet           fibrosa 
             0.28              0.28              0.28              0.28 
            finit           follicl               ger             h3dna 
             0.28              0.28              0.28              0.28 
            h3tdr        hypocalcem            indica            liabil 
             0.28              0.28              0.28              0.28 
            minal       normocalcem           osteiti     plasmacalcium 
             0.28              0.28              0.28              0.28 
       progenitor            qualli         radioacti            rentli 
             0.28              0.28              0.28              0.28 
           ricket            sacrif            sugino               thi 
             0.28              0.28              0.28              0.28 
         thymocyt         thymustyp        vertheless           statist 
             0.28              0.28              0.28              0.25

In [39]:
library(wordcloud)

Loading required package: RColorBrewer


In [40]:
m=as.matrix(dtmMedCollection)

In [41]:
v=sort(colSums(m),decreasing=TRUE)

In [42]:
myNames = names(v)

In [43]:
d=data.frame(word=myNames,freq=v)

In [45]:
wordcloud(d$word,d$freq,min.freq=2)

![png](wordcount.png)

## Conclusion

Since the support included (0.85) yielded no results, a smaller support bound was used. This implies that the relative occurence of a word in relation to another word was much smaller. The image included above illustrates and highlights the most likely occuring words. 

A random assumption utiliing the likelihood occurence may imply that the documents parsed described biological experiments which aimed to identify how cells interacted with other agents.